In [ ]:
# sudo -H anaconda/bin/pip install biopython

from Bio import Entrez
from Bio import SeqIO

In [ ]:
Entrez.email = 'greg.clinton@gmail.com'

In [ ]:
# http://www.ncbi.nlm.nih.gov/Sitemap/samplerecord.html

# How many genes does e. coli have? What are their names?
# How many species under homo?
# sra vs nucleotide, gene, ref_seq

In [3]:
q = 'human[porg] AND tp53[gene] AND biomol_rna[prop] AND srcdb_refseq[prop]' 
db = 'nuccore'

In [ ]:
q = 'human[porg] AND tp53[gene] AND biomol_rna[prop] AND srcdb_refseq[prop]' 
record = Entrez.read(Entrez.esearch(db = dv, term = q))
ids = ','.join(record['IdList'][:5])
handle = Entrez.efetch(db = 'nuccore', id = ids, rettype = 'gb', retmode = 'text')
records = SeqIO.parse(handle, 'gb')

In [ ]:
for record in records:
    print record.name

In [113]:
import requests
from lxml import etree

email = 'greg.clinton@gmail.com'

def entrez(cgi, path):
    url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/'
    url += cgi + '.fcgi?'
    url += '' if path == None else ('%s&' % path) 
    url += 'tool=entrez&email=' + email
    encoding = 'utf-8'
    xml = requests.get(url).text.encode(encoding)
    parser = etree.XMLParser(encoding = encoding)
    return etree.fromstring(xml, parser = parser)

def esearch(db, term):
    return entrez('esearch', 'db=%s&term=%s' % (db, term))

class Db: pass
class Field: pass

def einfo(db = None):
    root = entrez('einfo', '' if db == None else 'version=2.0&db=%s' % db)[0]
    if db == None:
        return [e.text for e in root]
    else:
        res = Db()
        
        for e in root:
            if e.tag == 'FieldList':
                fields = []
                for field in e:
                    info = Field()
                    for x in field:
                        setattr(info, x.tag, x.text)
                    fields.append(info)
                setattr(res, e.tag, fields)
            else:
                setattr(res, e.tag, e.text)
        return res
        
def databases():
    return einfo()

def fields(db):
    return einfo(db).FieldList